# Preprocessing pipeline

## Table of Contents
1. [Libraries](#libraries)
2. [Change format](#change-format)
4. [Preprocessing](#preprocessing)
5. [Save output](#save-output)


## 1. Libraries
Install and load necessary libraries.

In [ ]:
#Install necessary files
!pip install gpxpy
!pip install tcxreader
!pip install openpyxl

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gpxpy
import gpxpy.gpx
import tcxreader
import tcxreader.tcxreader
import openpyxl
import os
import shutil


## 2. Change format
Convert GPX and TCX to XLS format. In order to correctly analyse all the data its format should be firstly unified.

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Or upload local dataset
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip /content/drive/MyDrive/opencampus_all_files/Sport1.zip -d /content

In [ ]:
def convert_gpx_to_excel(gpx_file_path, output_file_path):
  # Initialize a DataFrame to store data
  all_data = []

  # Parse the GPX file
  with open(gpx_file_path, 'r') as gpx_file:
      gpx = gpxpy.parse(gpx_file)

  # Extract data (latitude, longitude, elevation, time, etc.)
  for track in gpx.tracks:
      for segment in track.segments:
          for point in segment.points:
              # Convert timezone-aware datetime to timezone-naive
              naive_time = point.time.replace(tzinfo=None) if point.time else None

              all_data.append({
                  'Latitude': point.latitude,
                  'Longitude': point.longitude,
                  'Elevation': point.elevation,
                  'Time': naive_time
              })

  # Convert the data into a DataFrame
  df = pd.DataFrame(all_data)

  # Write the DataFrame to an Excel file
  df.to_excel(output_file_path, index=False)


In [ ]:
# TCX to XLSX
def convert_tcx_to_excel(tcx_file_path, output_file_path):
    """
    Converts a TCX file to an Excel file with trackpoint data.

    Parameters:
    - tcx_file_path: str, path to the input TCX file
    - output_file_path: str, path to save the output Excel file
    """
    # Initialize the TCX reader
    tcx_reader = TCXReader()

    # Read the TCX file
    data: TCXExercise = tcx_reader.read(tcx_file_path)

    # List to store the trackpoint data
    trackpoint_data = []

    # Loop through all trackpoints and extract relevant information
    for trackpoint in data.trackpoints:
        trackpoint_data.append({
            'Time': trackpoint.time,
            'Latitude': trackpoint.latitude,
            'Longitude': trackpoint.longitude,
            'Elevation': trackpoint.elevation,
            'Distance': trackpoint.distance,
            'Heartrate': trackpoint.hr_value,
            'Cadence': trackpoint.cadence,
            'Speed': trackpoint.tpx_ext['Speed']
        })

    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(trackpoint_data)

    # Save the DataFrame to an Excel file
    df.to_excel(output_file_path, index=False, engine='openpyxl')


In [ ]:
# Define the directories
sport_dir = "/content/Sport"
sport_xlsx_dir = "/content/Sport_xlsx"

# Create Sport_xlsx directory if it doesn't exist
if not os.path.exists(sport_xlsx_dir):
    os.makedirs(sport_xlsx_dir)

# Function to convert files in a folder
def convert_files_in_folder(rider_folder):
    rider_xlsx_folder = os.path.join(sport_xlsx_dir, rider_folder)

    # Create the rider folder in Sport_xlsx directory if it doesn't exist
    if not os.path.exists(rider_xlsx_folder):
        os.makedirs(rider_xlsx_folder)

    rider_folder_path = os.path.join(sport_dir, rider_folder)

    # Process .gpx files
    gpx_files = [f for f in os.listdir(rider_folder_path) if f.endswith('.gpx')]
    for gpx_file in gpx_files:
        convert_gpx_to_excel(os.path.join(rider_folder_path, gpx_file),
                           os.path.join(rider_xlsx_folder, gpx_file.replace('.gpx', '.xlsx')))

    # Process .tcx files
    tcx_files = [f for f in os.listdir(rider_folder_path) if f.endswith('.tcx')]
    for tcx_file in tcx_files:
        convert_tcx_to_excel(os.path.join(rider_folder_path, tcx_file),
                           os.path.join(rider_xlsx_folder, tcx_file.replace('.tcx', '.xlsx')))


In [ ]:
Riders = ['Rider1', 'Rider2', 'Rider3', 'Rider4', 'Rider5', 'Rider6', 'Rider7', 'Rider8', 'Rider9']
for rider in Riders:
  convert_files_in_folder(rider)

### 3. Preprocessing
Necessary steps before training model on the data:
- Delete files recorded outside Europe
- Repair corrupted file (f97)
- Convert timestamp to seconds

Optional feature engineering:
- Calculate distance since beginning


In [ ]:
# Load data from the drive
!zip -r /content/Sport_xlsx/Rider1.zip /content/Sport_xlsx/Rider1

In [ ]:
# Preprocessing pipeline
import pandas as pd
import os
from geopy.distance import geodesic
from pyproj import Transformer

def preprocess_rider_data(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".xlsx"):
            filepath = os.path.join(input_dir, filename)
            try:
                df = pd.read_excel(filepath)

                # Step 1: If file is f97.xlsx, delete the first 11 rows
                if filename == "f97.xlsx":
                    df = df.iloc[11:].reset_index(drop=True)

                # Step 2: Delete the file if max_longitude < 10 or max_latitude < 40
                if df['Longitude'].max() < 10 or df['Latitude'].max() < 40:
                    print(f"File {filename} deleted due to coordinates.")
                    os.remove(filepath)
                    continue  # Skip the rest of the loop for this file

                # Step 3: Convert timestamp to seconds
                df['Time'] = pd.to_datetime(df['Time'])
                df['Time'] = (df['Time'] - df['Time'].iloc[0]).dt.total_seconds()

                # Step 4: Calculate distance since the beginning
                distances = []
                cumulative_distance = 0
                for i in range(len(df)):
                    if i > 0:
                        previous_coords = (df['Latitude'].iloc[i-1], df['Longitude'].iloc[i-1])
                        current_coords = (df['Latitude'].iloc[i], df['Longitude'].iloc[i])
                        distance = geodesic(previous_coords, current_coords).meters
                        cumulative_distance += distance
                    else:
                        distance = 0
                    distances.append(cumulative_distance)
                df['Distance'] = distances

                # Step 5: Save the preprocessed file
                output_filepath = os.path.join(output_dir, filename)
                df.to_excel(output_filepath, index=False)
                print(f"Processed and saved: {filename}")
            except Exception as e:
                print(f"Error processing {filename}: {e}")


# Example usage (replace with your actual paths):
input_directory = '/content/content/Sport_xlsx/Rider1/'
output_directory = '/content/content/Sport_xlsx/Rider1_preprocessed/'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

preprocess_rider_data(input_directory, output_directory)

# 4. Save output
Initial preprocessing output exported and saved to drive for the further utilization.

In [ ]:
!zip -r /content/content/Sport_xlsx/Rider1_preprocessed.zip /content/content/Sport_xlsx/Rider1_preprocessed